<a href="https://colab.research.google.com/github/ZhangYHe/MyColabDL_Repo/blob/main/work2_%E5%BC%A0%E9%9B%B2%E8%B5%AB_loss_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##损失函数
损失函数选择为交叉熵损失函数。   
参数y_hat为预测的样本在10个类别上的预测概率,y为每个样本的真实类别数字。通过y索引得到预测样本在真实类别上的预测概率，取负对数，即为损失值。

In [ ]:
def CE_loss(y_hat, y):
    return - torch.log(y_hat[range(len(y_hat)), y])


##Res-net 50网络

In [18]:
import torch
from torch import nn
from torch.nn import functional as F


class Residual(nn.Module):  
    def __init__(self, input_channels, num_channels_1, num_channels_2,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels_1,
                               kernel_size=1, padding=0, stride=1)
        self.conv2 = nn.Conv2d(num_channels_1, num_channels_1,
                               kernel_size=3, padding=1, stride=1)
        self.conv3 = nn.Conv2d(num_channels_1, num_channels_2,
                               kernel_size=1, padding=0, stride=1)
        # #self.conv4 = nn.Conv2d(input_channels, num_channels_1*4,
        #                        kernel_size=1, padding=0, stride=1)
        if use_1x1conv:
            self.conv4 = nn.Conv2d(input_channels, num_channels_2,
                                   kernel_size=1, stride=strides)
        else:
            self.conv4 = None
        self.bn1 = nn.BatchNorm2d(num_channels_1)
        self.bn2 = nn.BatchNorm2d(num_channels_1)
        self.bn3 = nn.BatchNorm2d(num_channels_2)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = F.relu(self.bn2(self.conv2(X)))
        Y = self.bn3(self.conv3(Y))
        if self.conv4:
            X = self.conv4(X)
        Y += X
        return F.relu(Y)


def resnet_block(input_channels, num_channels_1, num_residuals,first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels_1,num_channels_1*4,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels_1*4, num_channels_1,num_channels_1*4))
    return blk

b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
b2 = nn.Sequential(*resnet_block(64, 64, 3, first_block=True))
b3 = nn.Sequential(*resnet_block(256, 128, 4))
b4 = nn.Sequential(*resnet_block(512, 256, 6))
b5 = nn.Sequential(*resnet_block(1024, 512, 3))

net = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Linear(2048, 10))

#检验输出
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])


RuntimeError: ignored

##Res-net 34网络

In [20]:
import torch
from torch import nn
from torch.nn import functional as F


class Residual(nn.Module):  
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

def resnet_block(input_channels, num_channels, num_residuals,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
b2 = nn.Sequential(*resnet_block(64, 64, 3, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 4))
b4 = nn.Sequential(*resnet_block(128, 256, 6))
b5 = nn.Sequential(*resnet_block(256, 512, 3))

net = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Linear(512, 10))

#检验输出
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 128, 28, 28])
Sequential output shape:	 torch.Size([1, 256, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])
